# Application On all sources

## Imports

In [6]:
%load_ext autoreload
%autoreload 2
import numpy as np 
from matplotlib import pyplot as plt 
import pandas as pd 
import seaborn as sns
import joblib


In [12]:
from utilities import make_model
from choices import param_dict 
from utilities import deets

## Load

### Load Model

In [11]:
model = joblib.load('models/lightGBM-final.pkl')
#model
clf = model.clf
clf 

LGBMClassifier(class_weight={'AGN': 1.2524978967563674,
                             'CV': 25.745323836808723,
                             'HMXB': 2.0562192560551287,
                             'LMXB': 43.41041115489145,
                             'PULSAR': 208.24450024014098,
                             'STAR': 1.2132045326966958,
                             'ULX': 12.87770640060345,
                             'YSO': 1.5988527422270495},
               force_col_wise=True, is_unbalance=True, metric=['auc_mu'],
               num_class=8, objective='multiclass', random_state=42,
               sparse=True, verbosity=0)

### Load Data

In [33]:
x = pd.read_csv('compiled_data_v3/x_phot_minmax.csv' , index_col='name')
x_col = x.columns.to_list()
i = pd.read_csv('compiled_data_v3/id_frame.csv' , index_col='name')
df = pd.merge(i,x , left_index =True , right_index =True )
df = df[df['offset']<1.01]
feat_to_drop = np.unique(param_dict['hardness'] + param_dict['IRAC']+[
    '0p5_2csc' , '2-10 keV (XMM)' , '1_2_csc' , '0p5_8_csc'
]+['var_inter_hard_prob_hs', 'ks_intra_prob_b', 'var_inter_hard_sigma_hm', 'var_inter_hard_prob_ms', 'var_inter_hard_prob_hm',] )
classes = ['AGN' ,'STAR' ,'YSO' ,'HMXB' , 'LMXB' , 'ULX' ,'CV' , 'PULSAR']
df = df[df['class'].isin(classes)]
#from sklearn.preprocessing import LabelEncoder
#y_int = LabelEncoder().fit_transform(df['class'])
#deets(df,1)
u_var = pd.read_csv('compiled_data_v3/x_phot_minmax-unid-var-src.csv' , index_col='name')
u_var = u_var.drop(columns=feat_to_drop)
ui_var = pd.read_csv('compiled_data_v3/id_frame-unid-var-src.csv' , index_col='name')
u_var.columns = u_var.columns.str.replace(' ' ,'_' , regex=False)
x = df[x_col]
x = x.drop(columns=feat_to_drop)
x.columns = x.columns.str.replace(' ' ,'_' , regex=False)
y = df['class']
deets(x)
deets(u_var)

_____________________________________________________
------------------------------
Number of Objects : 7703
Number of Columns : 41
_____________________________________________________
_____________________________________________________
------------------------------
Number of Objects : 39893
Number of Columns : 41
_____________________________________________________


In [34]:
u = []
ui = []
for i in range(8):
    u.append(pd.read_csv(f'compiled_data_v3/unid-non-var-chunks/x-chunk-{i}.csv' , index_col='name'))
    ui.append(pd.read_csv(f'compiled_data_v3/unid-non-var-chunks/id-chunk-{i}.csv' , index_col='name'))
u.append(u_var)
ui.append(ui_var)
u = pd.concat(u)
ui = pd.concat(ui)

#ui = pd.read_csv('compiled_data_v3/id_frame-unid-var-src.csv' , index_col='name')
#u = pd.read_csv('compiled_data_v3/x_phot_minmax-unid-var-src.csv' , index_col='name')
u = u.drop(columns=feat_to_drop)
u.columns = u.columns.str.replace(' ' ,'_' , regex=False)
u
u = u.reset_index()
# remove sources which were in training set
u = u[~u['name'].isin(x.index.to_list())].set_index('name')
u

,gal_b2,gal_l2,var_inter_prob_b,var_inter_sigma_b,var_intra_prob_b,var_inter_index_b,kp_intra_prob_b,var_intra_index_b,h-csc,b-csc,...,B-R,G-J,G-W2,Bp-H,Bp-W3,Rp-K,J-H,J-W1,W1-W2,24_microns_(MIPS)
name,,,,,,,,,,,,,,,,,,,,,
2CXO J003935.9-732725,2.561951,8.447115,NaN,NaN,2.809762,NaN,8.723285,0.0,4.972998e-06,0.003956,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2CXO J003936.7-731249,2.548397,8.447672,NaN,NaN,5.267622,NaN,3.246585,0.0,NaN,0.016586,...,2.628340,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2CXO J004028.7-731106,2.546591,8.445331,NaN,NaN,2.001693,NaN,2.474462,0.0,NaN,0.124419,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2CXO J004506.3-730056,2.536271,8.432664,NaN,NaN,1.712439,NaN,8.102221,0.0,1.135905e-08,0.027585,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2CXO J004659.0-731918,2.553126,8.427077,NaN,NaN,2.771679,NaN,3.443930,0.0,7.595018e-09,0.004986,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2CXO J220433.1-495916,2.153631,9.602391,6.284069,0.008090,0.848783,5.00,9.398184,0.0,2.267642e-07,0.035677,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2CXO J220508.3-495935,2.148799,9.600738,7.232708,0.007515,7.897757,6.25,9.373067,2.0,8.028243e-08,0.041673,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2CXO J220544.0-495505,2.142337,9.602388,7.905990,0.007822,NaN,6.25,NaN,NaN,1.863801e-07,0.016260,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [35]:
ui

,significance,var_flag,streak_src_flag,pileup_flag,ra,dec,missing,sat_src_flag,extent_flag,conf_flag
name,,,,,,,,,,
2CXO J003935.9-732725,3.294118,0,0,0,9.899767,-73.457136,0.576271,0,0,0
2CXO J003936.7-731249,4.157895,0,0,0,9.903193,-73.213742,0.525424,0,0,0
2CXO J004028.7-731106,14.422175,0,0,0,10.119748,-73.185171,0.508475,0,0,0
2CXO J004506.3-730056,10.447263,0,0,0,11.276413,-73.015666,0.593220,0,0,0
2CXO J004659.0-731918,2.444444,0,0,0,11.745895,-73.321890,0.593220,0,0,0
...,...,...,...,...,...,...,...,...,...,...
2CXO J220433.1-495916,2.651163,1,0,0,331.138121,-49.987838,0.491525,0,0,0
2CXO J220508.3-495935,4.965517,1,0,0,331.284642,-49.993301,0.474576,0,0,0
2CXO J220544.0-495505,2.857143,1,0,0,331.433541,-49.918280,0.542373,0,0,0


## Make Predictions

In [22]:
from utilities import softmax , norm_prob
pred_prob = (clf.predict_proba(u))
pred_prob_df = pd.DataFrame(pred_prob , columns=[f'prob_{el}' for el in clf.classes_] , index = u.index.to_list())
pred_prob_df

,prob_AGN,prob_CV,prob_HMXB,prob_LMXB,prob_PULSAR,prob_STAR,prob_ULX,prob_YSO
2CXO J003935.9-732725,0.060166,0.736604,0.008820,2.893247e-04,0.017486,0.165225,0.011179,0.000231
2CXO J003936.7-731249,0.090444,0.008151,0.016982,7.988541e-05,0.865511,0.017638,0.001115,0.000080
2CXO J004028.7-731106,0.064571,0.005341,0.207369,7.960503e-05,0.002110,0.010129,0.710191,0.000210
2CXO J004506.3-730056,0.031245,0.158560,0.026846,1.074152e-04,0.182130,0.035309,0.565726,0.000077
2CXO J004659.0-731918,0.007044,0.000213,0.410738,4.800778e-05,0.057428,0.492584,0.031912,0.000033
...,...,...,...,...,...,...,...,...
2CXO J220556.3-500415,0.061626,0.003023,0.925260,2.713142e-05,0.005887,0.002594,0.001563,0.000021
2CXO J220557.3-500827,0.000259,0.007889,0.954209,4.198581e-06,0.036613,0.000088,0.000931,0.000006
2CXO J220600.9-501131,0.001970,0.009915,0.369751,5.611201e-05,0.616852,0.000227,0.001218,0.000010
2CXO J220601.0-501116,0.000166,0.000098,0.998545,7.927362e-07,0.000508,0.000013,0.000667,0.000002


In [23]:
u_df = pd.DataFrame({
    'name' : u.index.to_list() , 
    'class' : clf.predict(u) , 
    'prob' : [np.amax(el) for el in pred_prob] ,
    'prob_margin' : [el[-1]-el[-2] for el in np.sort(pred_prob , axis=1 ,)]
}).set_index('name')
u_df = pd.merge(u_df , pred_prob_df , left_index=True , right_index=True)
u_df.index.name = 'name'
u_df 

,class,prob,prob_margin,prob_AGN,prob_CV,prob_HMXB,prob_LMXB,prob_PULSAR,prob_STAR,prob_ULX,prob_YSO
name,,,,,,,,,,,
2CXO J003935.9-732725,CV,0.736604,0.571379,0.060166,0.736604,0.008820,2.893247e-04,0.017486,0.165225,0.011179,0.000231
2CXO J003936.7-731249,PULSAR,0.865511,0.775067,0.090444,0.008151,0.016982,7.988541e-05,0.865511,0.017638,0.001115,0.000080
2CXO J004028.7-731106,ULX,0.710191,0.502821,0.064571,0.005341,0.207369,7.960503e-05,0.002110,0.010129,0.710191,0.000210
2CXO J004506.3-730056,ULX,0.565726,0.383596,0.031245,0.158560,0.026846,1.074152e-04,0.182130,0.035309,0.565726,0.000077
2CXO J004659.0-731918,STAR,0.492584,0.081845,0.007044,0.000213,0.410738,4.800778e-05,0.057428,0.492584,0.031912,0.000033
...,...,...,...,...,...,...,...,...,...,...,...
2CXO J220556.3-500415,HMXB,0.925260,0.863634,0.061626,0.003023,0.925260,2.713142e-05,0.005887,0.002594,0.001563,0.000021
2CXO J220557.3-500827,HMXB,0.954209,0.917596,0.000259,0.007889,0.954209,4.198581e-06,0.036613,0.000088,0.000931,0.000006
2CXO J220600.9-501131,PULSAR,0.616852,0.247101,0.001970,0.009915,0.369751,5.611201e-05,0.616852,0.000227,0.001218,0.000010


## Result Analysis

In [25]:
u_df_num = u_df['class'].value_counts().to_frame()
th_list = [0.5 , 0.8 , 0.997]
u_df_num = pd.concat([
                u_df['class'].value_counts() ,
                #u_df[u_df['prob']>mode]['class'].value_counts(),
                u_df[u_df['prob']>0.5]['class'].value_counts(),
                #u_df[u_df['prob']>0.6]['class'].value_counts(),
                #u_df[u_df['prob']>0.7]['class'].value_counts(),
                u_df[u_df['prob']>0.8]['class'].value_counts(),
                #u_df[u_df['prob']>0.9]['class'].value_counts(),
                u_df[u_df['prob']>0.997]['class'].value_counts(),
                 ] , axis=1)
u_df_num.columns = ['Argmax'  ,'0.5' , '0.8'  ,  f"0.997"]
u_df_num.index.name = 'class'
#u_df_num.to_csv(f'temp_res_comp/unid_num/{d_name}_{m_name}.csv')
u_df_num.append(u_df_num.sum() , ignore_index=True)
print(u_df_num.to_markdown())

| class   |   Argmax |   0.5 |   0.8 |   0.997 |
|:--------|---------:|------:|------:|--------:|
| AGN     |    82549 | 78492 | 62112 |   26512 |
| STAR    |    39502 | 37080 | 28161 |   11562 |
| PULSAR  |    25492 | 21859 | 11759 |     389 |
| HMXB    |    25029 | 21849 | 13047 |    2001 |
| ULX     |    23591 | 20662 | 10874 |     428 |
| YSO     |    23165 | 22106 | 15696 |    1541 |
| CV      |    11523 |  9200 |  4169 |     201 |
| LMXB    |      642 |   627 |   551 |     243 |


In [26]:
from numpy.random import normal
from ridgeplot import ridgeplot
cl = classes[::-1]
#cl.remove('LMXB')
sample = [u_df[u_df['class']==c]['prob'] for c in cl]
fig = ridgeplot(
    samples=sample,
    kde_points=np.linspace(0, 1, 500),
    colorscale="inferno",
    colormode="index",
    coloralpha=0.6,
    labels=cl,
    spacing = 3 / 8,
    xpad=0
)
fig.update_layout(
    title="Unidentified sources | No Imputation | LightGBM classifier",
    height=550,
    width=600,
    plot_bgcolor="rgba(255, 255, 255, 0.0)",
    xaxis_gridcolor="rgba(0, 0, 0, 0.1)",
    yaxis_gridcolor="rgba(0, 0, 0, 0.1)",
    xaxis_title="Predicted Probability",
    yaxis_title="Predicted Probability Density",
)

#fig.save('temp.png')
#fig.write_image('temp_res_comp/unid_prob_dist/lightGBM_none.jpg')
fig.show()